<a href="https://colab.research.google.com/github/Hamxea/Bidirectional-Encoder-Representations-from-Transformers-Turkish-Text-Classification/blob/main/cmp711project_baseline_nlp_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding
import pandas as pd
from tensorflow.python.keras.layers import GRU, Bidirectional
from tensorflow.python.keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

from google.colab import drive

import tensorflow as tf

In [4]:
# set environment as googledrive to folder "resource"
data_path =  "/dataset/"

try:
    drive.mount('/content/drive')
    data_path = "/content/drive/My Drive/dataset/"

except:
    print("You are not working in Colab at the moment :(")

Mounted at /content/drive


In [5]:
df = pd.read_csv(data_path + 'turkish_text_data.csv', sep=';', encoding='utf-8')

In [6]:
df.drop_duplicates(subset=['text'], keep='first', inplace=True)

df = df.sample(frac=1)
df.reset_index(drop=True, inplace=True)

print(df.groupby('category').count())
print(df.shape)

           text
category       
dunya       677
ekonomi     690
kultur      567
saglik      632
siyaset     690
spor        636
teknoloji   647
(4539, 2)


In [7]:
df.head(10)

,category,text
0,saglik,isınayım derken gerilmeyin sakarya_üniversites...
1,ekonomi,asya borsaları karışık seyretti asya borsaları...
2,dunya,gizli belgeleri gözardı etmeyeceğiz suriye muh...
3,teknoloji,chrome 24 yayınlandı google web tarayıcısının ...
4,siyaset,demirtaş resepsiyona katılacak mı bdp eş genel...
5,saglik,ultrason zararsızdır denilemez art tıp merkezi...
6,saglik,gribe karşı limon ve bal ege_üniversitesi tıp_...
7,siyaset,bio ajanlık aracısı haline gelebilir chp_izmir...
8,siyaset,atalay kılıçdaroğlu risk alsın başbakan_yardım...
9,ekonomi,sigaracılardan yatırım tehdidi tütün ve alkol_...


### Clean the Train data, specifically remove punctuations

In [8]:
import string
import re

totalContentCleaned = []
punctDict = {}
for punct in string.punctuation:
    punctDict[punct] = None
transString = str.maketrans(punctDict)
# since we intent to remove any punctuation with ''
for sen in df['text']:
    
    cleanedString = re.sub('[^a-zA-Z]+ ', '', sen)
    
    p = cleanedString.translate(transString)
    totalContentCleaned.append(p)
    

df['text'] = totalContentCleaned

In [9]:
output_dim = df.category.unique().size

target = df['category'].values.tolist()
data = df['text'].values.tolist()

encoder = LabelEncoder()
encoder.fit(target)
encoded_target = encoder.transform(target)
# convert integers to one hot encoded vectors
y_train = np_utils.to_categorical(encoded_target)
X_train = data

num_words= 12000
tokenizer = Tokenizer(num_words=num_words)

tokenizer.fit_on_texts(data)
tokenizer.word_index
print("Total vocab size:", len(tokenizer.word_index))


Total vocab size: 110951


In [10]:
# The Tokenizer stores everything in the word_index during fit_on_texts. Then, when calling the texts_to_sequences method, only the top num_words are considered
tokenizer.index_word[12000]

X_train_tokens = tokenizer.texts_to_sequences(X_train)

X_train[800]
print(X_train_tokens[800])

[383, 152, 91, 982, 67, 1651, 222, 3320, 1560, 383, 152, 4313, 3563, 522, 7304, 982, 3837, 226, 178, 278, 342, 383, 10864, 67, 982, 3505, 6550, 10152, 1, 4313, 3563, 7842, 2497, 982, 7, 2, 4046, 269, 41, 537, 4477, 4313, 3563, 522, 7304, 1808, 4296, 98, 4296, 4525, 24, 712, 1286, 2758, 982, 944, 234, 106, 169, 762, 433, 1084, 2831, 2620, 192, 1710, 3, 59, 1705, 7900, 2, 660, 91, 982, 24, 178, 404, 1416, 226, 178, 404, 54, 124, 2, 383, 9591, 5100, 11521, 4313, 3563, 522, 2, 8999, 982, 1572, 234, 226, 712, 1371, 1591, 1997, 755, 2, 2119, 1170, 2391, 383, 72, 4168, 41, 3, 7065, 24, 178, 12, 1651, 1835, 6000, 637, 129, 1394, 10035, 41, 383, 72, 982, 304, 433, 1084, 2831, 2620, 3, 1553, 2, 553, 544, 2002, 278, 383, 409, 1050, 40, 334, 2, 481, 433, 7303]


In [11]:
num_tokens = [len(tokens) for tokens in X_train_tokens]
num_tokens = np.array(num_tokens)

np.mean(num_tokens)
np.max(num_tokens)
np.argmax(num_tokens) # index gösteriyor

X_train[np.argmax(num_tokens)]

# max_tokens belirleme
max_tokens = int(np.mean(num_tokens) + (2 * np.std(num_tokens)))
max_tokens

print("What percentage of the total does max_token contain: %", int(100 * np.sum(num_tokens < max_tokens) / len(num_tokens)))

What percentage of the total does max_token contain: % 95


In [12]:
X_train_pad = pad_sequences(X_train_tokens, maxlen=max_tokens)
X_train_pad.shape

np.array(X_train_tokens[800])
X_train_pad[800]

idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(), idx.keys()))

def tokens_to_string(tokens):
    words = [inverse_map[token] for token in tokens if token!=0]
    text = ' '.join(words)
    return text

tokens_to_string([1,2,3,4])
X_train[800]
X_train_tokens[800]
tokens_to_string(X_train_tokens[800])

'facebook ta özel mesaj önce dikkatli sosyal ağ sitesi facebook ta arkadaş listesinde olmayan birine mesaj atmak 100 dolar olabilir mi facebook haftalar önce mesaj sisteminde değişikliğe gideceğini ve arkadaş listesinde bulunmayan kişilere mesaj için bir açıklamıştı buna göre normal şartlarda arkadaş listesinde olmayan birine atılan mesajlar diğer mesajlar giderken 1 dolarlık ücret karşılığında mesaj doğrudan kişinin gelen na sistem henüz test aşamasında olduğundan birçok kullanıcı bu ancak internette dolaşan bir fotoğraf özel mesaj 1 dolar mı yoksa 100 dolar mı olduğu konusunda bir facebook kullanıcısının paylaştığı fotoğrafta arkadaş listesinde olmayan bir kullanıcıya mesaj atan kişinin 100 dolarlık ödeme yapması gerektiğine dair bir uyarı çıkıyor oysa facebook un açıklamasına göre bu bedel 1 dolar olarak dikkatli bakıldığında fotoğrafın olmadığı dikkat çekiyor uzmanlara göre facebook un mesaj sistemi henüz test aşamasında olduğundan bu tip bir sorun meydana gelmiş olabilir facebook 

In [13]:
def baseline_model():

    model = Sequential()
    embedding_size = 100

    model.add(Embedding(input_dim=num_words,
                        output_dim=embedding_size,
                        input_length=max_tokens,
                        name='embedding_layer'))

    model.add(Bidirectional(GRU(units=250, return_sequences=True)))
    model.add(Dropout(0.3))
    model.add(Bidirectional(GRU(units=250)))
    model.add(Dense(output_dim, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [14]:
print(output_dim)

7


In [15]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=10, batch_size=128, verbose=1)
kfold = KFold(n_splits=2, shuffle=True)
results = cross_val_score(estimator, X_train_pad, y_train, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Epoch 1/10
18/18 [==============================] - 134s 7s/step - loss: 1.9417 - accuracy: 0.1678
Epoch 2/10
18/18 [==============================] - 123s 7s/step - loss: 1.6998 - accuracy: 0.4132
Epoch 3/10
18/18 [==============================] - 125s 7s/step - loss: 0.9296 - accuracy: 0.6760
Epoch 4/10
18/18 [==============================] - 126s 7s/step - loss: 0.2840 - accuracy: 0.9170
Epoch 5/10
18/18 [==============================] - 124s 7s/step - loss: 0.0587 - accuracy: 0.9881
Epoch 6/10
18/18 [==============================] - 125s 7s/step - loss: 0.0143 - accuracy: 0.9984
Epoch 7/10
18/18 [==============================] - 124s 7s/step - loss: 0.0038 - accuracy: 0.9997
Epoch 8/10
18/18 [==============================] - 127s 7s/step - loss: 0.0032 - accuracy: 1.0000
Epoch 9/10
18/18 [==============================] - 124s 7s/step - loss: 8.3562e-04 - accuracy: 1.0000
Epoch 10/10
18/18 [==============================] - 12s 627ms/step - loss: 2.1822 - accuracy: 0.6498
Epo

In [16]:
precision = cross_val_score(estimator, X_train_pad, y_train, cv=kfold, scoring='precision')
f1 = cross_val_score(estimator, X_train_pad, y_train, cv=kfold, scoring='f1')

In [ ]:
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
print('Recall', np.mean(recall), recall)
print('Precision', np.mean(precision), precision)
print('F1', np.mean(f1), f1)

In [ ]:
print("Accuracy: %.2f%% (%.2f%%)" % (recall.mean()*100, recall.std()*100))